In [42]:
import torch
import random
import numpy as np
import re

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

PATH="/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity/Sampling/Masking"
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool
FAST_ALIGN="/home/vzhekova/fast_align/build/fast_align" # path to the fast_align tool
TERCOM = "/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE"

In [43]:
# check if we can connect to the GPU with PyTorch
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

Current device: GeForce GTX 1080 Ti


In [44]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity/Sampling/Masking


# Translation English-German

In [4]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"

In [ ]:
# Generate translations
# Beam search
!fairseq-generate data-bin_original_man_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_man_en-de.decode_Beam_10.log

!fairseq-generate data-bin_original_woman_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_woman_en-de.decode_Beam_10.log

!fairseq-generate data-bin_original_girl_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_girl_en-de.decode_Beam_10.log

!fairseq-generate data-bin_original_guy_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_guy_en-de.decode_Beam_10.log

!fairseq-generate data-bin_original_boy_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_boy_en-de.decode_Beam_10.log

print('Finished translation.')

2023-10-07 14:43:35 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [7]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_man_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_man_50.txt
!grep ^H original_woman_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_woman_50.txt
!grep ^H original_girl_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_girl_50.txt
!grep ^H original_guy_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_guy_50.txt
!grep ^H original_boy_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_boy_50.txt


## Extract 10 unique sentences for every source

In [13]:
# List with nbest sentences for every source in original
nbest_original = []
counter = 0
temp = []
with open('hyp_original_boy_50.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 50):
            nbest_original.append(temp)
            counter = 0
            temp = []

#print(nbest_original)

sent_hyp = []
for elem in nbest_original:
    unique_sent = set()
    for sent in elem:
        unique_sent.add(sent)
        if (len(unique_sent) == 10):
            sent_hyp.extend(list(unique_sent))
            break
        
print (len(sent_hyp))

with open('hyp_original_boy.txt','w') as fout:
    for sent in sent_hyp:
        print(sent, end='\n', file=fout)

3350


# Backtranslation German-English

In [14]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.hyp_original_man.de hyp_original_man.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_original_woman.de hyp_original_woman.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_original_girl.de hyp_original_girl.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_original_guy.de hyp_original_guy.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_original_boy.de hyp_original_boy.txt bpecodes.de

print('Finished subword.')

Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original_man.txt ...
Read 25359 words (1793 unique) from text file.
Applying BPE to hyp_original_man.txt ...
Modified 25359 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original_woman.txt ...
Read 25514 words (1884 unique) from text file.
Applying BPE to hyp_original_woman.txt ...
Modified 25514 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original_girl.txt ...
Read 25455 words (1908 unique) from text file.
Applying BPE to hyp_original_girl.txt ...
Modified 25455 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original_guy.txt ...
Read 24946 words (1558 unique) from text file.
Applying BPE to hyp_original_guy.txt ...
Modified 24946 words from text file.
Loadin

In [15]:
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original_man \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_man_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original_woman \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_woman_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original_girl \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_girl_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original_guy \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_guy_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original_boy \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_boy_de-en \
    --workers 8

print('Finished preprocessing.')

2023-10-08 11:04:20 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_original_man_de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='de', 

In [16]:
MODELS="/export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble"

In [17]:
# Generate backtranslations
!fairseq-generate data-bin_original_man_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_man_de-en.decode_Beam_10_backtranslation.log

!fairseq-generate data-bin_original_woman_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_woman_de-en.decode_Beam_10_backtranslation.log

!fairseq-generate data-bin_original_girl_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_girl_de-en.decode_Beam_10_backtranslation.log

!fairseq-generate data-bin_original_guy_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_guy_de-en.decode_Beam_10_backtranslation.log

!fairseq-generate data-bin_original_boy_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --sampling \
    --beam 50 \
    --nbest 50 \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > original_boy_de-en.decode_Beam_10_backtranslation.log

print('Finished translation.')

2023-10-08 11:05:18 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [18]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_man_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_man_back_50.txt
!grep ^H original_woman_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_woman_back_50.txt
!grep ^H original_girl_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_girl_back_50.txt
!grep ^H original_guy_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_guy_back_50.txt
!grep ^H original_boy_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_boy_back_50.txt


## Extract 10 unique sentences for every source

In [26]:
# List with nbest sentences for every source in original
nbest_original = []
counter = 0
temp = []
with open('hyp_original_boy_back_50.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 50):
            nbest_original.append(temp)
            counter = 0
            temp = []

#print(nbest_original)

sent_hyp = []
for elem in nbest_original:
    unique_sent = set()
    for sent in elem:
        unique_sent.add(sent)
        if (len(unique_sent) == 10):
            sent_hyp.extend(list(unique_sent))
            break
        
print (len(sent_hyp))

with open('hyp_original_boy_back.txt','w') as fout:
    for sent in sent_hyp:
        print(sent, end='\n', file=fout)

33500


In [27]:
# Detokenize text        
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

md_en = MosesDetokenizer(lang='en')

with open('hyp_original_man_back.txt', encoding='utf8') as fin, open('original_man_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_original_woman_back.txt', encoding='utf8') as fin, open('original_woman_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_original_girl_back.txt', encoding='utf8') as fin, open('original_girl_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_original_guy_back.txt', encoding='utf8') as fin, open('original_guy_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_original_boy_back.txt', encoding='utf8') as fin, open('original_boy_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


# Statistics on translations

In [28]:
# List with original source sentences
source_man = []
with open('en_original_man.txt', 'r') as fin:
    for line in fin:
        source_man.append(line.strip())
        
source_woman = []
with open('en_original_woman.txt', 'r') as fin:
    for line in fin:
        source_woman.append(line.strip())
        
source_girl = []
with open('en_original_girl.txt', 'r') as fin:
    for line in fin:
        source_girl.append(line.strip())
        
source_guy = []
with open('en_original_guy.txt', 'r') as fin:
    for line in fin:
        source_guy.append(line.strip())
        
source_boy = []
with open('en_original_boy.txt', 'r') as fin:
    for line in fin:
        source_boy.append(line.strip())
    
# List with nbest sentences for every source in original
nbest_original_man = []
counter = 0
temp = []
with open('hyp_original_man.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_original_man.append(temp)
            counter = 0
            temp = []
            
nbest_original_woman = []
counter = 0
temp = []
with open('hyp_original_woman.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_original_woman.append(temp)
            counter = 0
            temp = []
            
nbest_original_girl = []
counter = 0
temp = []
with open('hyp_original_girl.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_original_girl.append(temp)
            counter = 0
            temp = []
            
nbest_original_guy = []
counter = 0
temp = []
with open('hyp_original_guy.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_original_guy.append(temp)
            counter = 0
            temp = []
            
nbest_original_boy = []
counter = 0
temp = []
with open('hyp_original_boy.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_original_boy.append(temp)
            counter = 0
            temp = []
        
            
print(len(source_man))
print(len(nbest_original_man))

335
335


## Count unique sentences

In [29]:
# Count unique sentences in source nbest list for each source sentence
def count_unique_sentences(nbest_sentences):
    unique_sent = []
    for source_nbest in nbest_sentences:
        num_values = len(set(source_nbest))
        #print(num_values)
        unique_sent.append(num_values)

    #print(unique_sent)
    return sum(unique_sent)/len(nbest_sentences) # average

In [30]:
# Value should be 10, because beam search generates 10 unique sentences
print(count_unique_sentences(nbest_original_man))
print(count_unique_sentences(nbest_original_woman))
print(count_unique_sentences(nbest_original_girl))
print(count_unique_sentences(nbest_original_guy))
print(count_unique_sentences(nbest_original_boy))
print("Average: ")
print((count_unique_sentences(nbest_original_man) + count_unique_sentences(nbest_original_woman) + 
      count_unique_sentences(nbest_original_girl) + count_unique_sentences(nbest_original_guy) + count_unique_sentences(nbest_original_boy))/5)

10.0
10.0
10.0
10.0
10.0
Average: 
10.0


## Count unique words

In [31]:
# Count unique words in source nbest list for each source sentence of original
# !!! Method is slow
import spacy

def count_unique_words(nbest_sentences):
    sp = spacy.load('en_core_web_sm')
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    unique_words = []
    normalizer = 0 # should normalize based on total number of words, because disambiguated sentences have more words overall
    counter = 0
    for source_nbest in nbest_sentences:
        words = set()
        for sent in source_nbest:
            tokens = sp(sent)
            normalizer += len(tokens)
            for token in tokens:
                # if token.text not in stopwords:    # checking whether the word is a stop word
                    words.add(token.text)
        num_values = len(words)
        unique_words.append(num_values)

        counter += 1
        #print(counter)

    #print(unique_words)
    print('Normalizer: ' + str(normalizer/len(nbest_sentences)))
    return (sum(unique_words)/len(nbest_sentences), (sum(unique_words)/len(nbest_sentences))/(normalizer/len(nbest_sentences))) # (average, norm average)

In [32]:
print(count_unique_words(nbest_original_man))
print(count_unique_words(nbest_original_woman))
print(count_unique_words(nbest_original_girl))
print(count_unique_words(nbest_original_guy))
print(count_unique_words(nbest_original_boy))

Normalizer: 75.83582089552239
(19.79402985074627, 0.2610116118874238)
Normalizer: 76.28955223880597
(20.08955223880597, 0.26333294205110147)
Normalizer: 76.14029850746269
(19.728358208955225, 0.25910534363116006)
Normalizer: 74.65074626865672
(20.044776119402986, 0.26851407549584133)
Normalizer: 75.75820895522388
(19.907462686567165, 0.26277631112336974)


# Statistics on backtranslations

In [33]:
# List with original source sentences
source_man = []
with open('en_original_man.txt', 'r') as fin:
    for line in fin:
        source_man.append(line.strip())
        
source_woman = []
with open('en_original_woman.txt', 'r') as fin:
    for line in fin:
        source_woman.append(line.strip())
        
source_girl = []
with open('en_original_girl.txt', 'r') as fin:
    for line in fin:
        source_girl.append(line.strip())
        
source_guy = []
with open('en_original_guy.txt', 'r') as fin:
    for line in fin:
        source_guy.append(line.strip())
        
source_boy = []
with open('en_original_boy.txt', 'r') as fin:
    for line in fin:
        source_boy.append(line.strip())
    
# List with nbest sentences for every source in original
nbest_original_man = []
counter = 0
temp = []
with open('original_man_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original_man.append(temp)
            counter = 0
            temp = []
            
nbest_original_woman = []
counter = 0
temp = []
with open('original_woman_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original_woman.append(temp)
            counter = 0
            temp = []
            
nbest_original_girl = []
counter = 0
temp = []
with open('original_girl_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original_girl.append(temp)
            counter = 0
            temp = []
            
nbest_original_guy = []
counter = 0
temp = []
with open('original_guy_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original_guy.append(temp)
            counter = 0
            temp = []
            
nbest_original_boy = []
counter = 0
temp = []
with open('original_boy_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original_boy.append(temp)
            counter = 0
            temp = []
        
            
print(len(source_man))
print(len(nbest_original_man))

335
335


## Source sentence reoccurrence

In [34]:
# Count how many of the source sentences reoccur in the backtranslation
def count_sentence_reoccurrence(source_sentences, nbest_sentences):
    results = []
    counter = 0
    for sent in source_sentences:
        matches = 0
        for target in nbest_sentences[counter]: 
            if (sent == target):
                matches += 1
        results.append(matches)  
        counter += 1

    return sum(x > 0 for x in results)

In [35]:
print(count_sentence_reoccurrence(source_man, nbest_original_man))
print(count_sentence_reoccurrence(source_woman, nbest_original_woman))
print(count_sentence_reoccurrence(source_girl, nbest_original_girl))
print(count_sentence_reoccurrence(source_guy, nbest_original_guy))
print(count_sentence_reoccurrence(source_boy, nbest_original_boy))
print("Average: ")
print((count_sentence_reoccurrence(source_man, nbest_original_man) + count_sentence_reoccurrence(source_woman, nbest_original_woman) + 
      count_sentence_reoccurrence(source_girl, nbest_original_girl) + count_sentence_reoccurrence(source_guy, nbest_original_guy) +
      count_sentence_reoccurrence(source_boy, nbest_original_boy))/5)

296
293
296
290
302
Average: 
295.4


## Ambiguous source words reoccurrence


In [36]:
# Count how many of the ambiguous words reoccur in the backtranslation
def count_words_reoccurrence(ambiguous_words, nbest_sentences):
    results = []
    counter = 0
    for word in ambiguous_words:
        matches = 0
        for target in nbest_sentences[counter]: 
            if (word in target.split(" ")):
                matches += 1
        results.append(matches)  
        counter += 1

    return sum(x > 0 for x in results)

In [37]:
print(count_words_reoccurrence(["man"] * 335, nbest_original_man))
print(count_words_reoccurrence(["woman"] * 335, nbest_original_woman))
print(count_words_reoccurrence(["girl"] * 335, nbest_original_girl))
print(count_words_reoccurrence(["guy"] * 335, nbest_original_guy))
print(count_words_reoccurrence(["boy"] * 335, nbest_original_boy))
print("Average: ")
print((count_words_reoccurrence(["man"] * 335, nbest_original_man) + count_words_reoccurrence(["woman"] * 335, nbest_original_woman) +
      count_words_reoccurrence(["girl"] * 335, nbest_original_girl) + count_words_reoccurrence(["guy"] * 335, nbest_original_guy) + 
      count_words_reoccurrence(["boy"] * 335, nbest_original_boy))/5)

335
335
335
333
335
Average: 
334.6


## Count unique sentences

In [38]:
print(count_unique_sentences(nbest_original_man))
print(count_unique_sentences(nbest_original_woman))
print(count_unique_sentences(nbest_original_girl))
print(count_unique_sentences(nbest_original_guy))
print(count_unique_sentences(nbest_original_boy))
print("Average: ")
print((count_unique_sentences(nbest_original_man) + count_unique_sentences(nbest_original_woman) + 
      count_unique_sentences(nbest_original_girl) + count_unique_sentences(nbest_original_guy) + count_unique_sentences(nbest_original_boy))/5)

80.55223880597015
80.7044776119403
80.74626865671642
82.0179104477612
78.94029850746269
Average: 
80.59223880597015


## Count unique words

In [39]:
print(count_unique_words(nbest_original_man))
print(count_unique_words(nbest_original_woman))
print(count_unique_words(nbest_original_girl))
print(count_unique_words(nbest_original_guy))
print(count_unique_words(nbest_original_boy))

Normalizer: 753.9791044776119
(76.57611940298507, 0.10156265465213414)
Normalizer: 754.289552238806
(73.12537313432836, 0.09694602413262257)
Normalizer: 764.3522388059702
(76.9134328358209, 0.10062563950354998)
Normalizer: 747.5641791044776
(70.71044776119403, 0.09458779558686121)
Normalizer: 754.4865671641791
(72.20298507462687, 0.09569817173287756)


# Word alignement (source-translation)

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [45]:
def build_alignment_input(sentencesN, sourceIn, targetIn, output):
    # List with original source sentences
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    # List with nbest sentences for every source in original 
    target = []
    counter = 0
    temp = []
    with open(targetIn, 'r') as fin:
        for line in fin:
            temp.append(line.strip())
            counter += 1
            if (counter == 10):
                target.append(temp)
                counter = 0
                temp = []

    #print(len(source))
    #print(len(target))           

    count = 0
    with open(output, 'w') as fout:
        while count < sentencesN:
            for hyp in target[count]:
                print(source[count] + ' ||| ' + hyp, end='\n', file=fout)
            count += 1

In [46]:
build_alignment_input(335, 'tok.en_original_man.en', 'hyp_original_man.txt', 'original_man_source-target_en-de.txt')
build_alignment_input(335, 'tok.en_original_woman.en', 'hyp_original_woman.txt', 'original_woman_source-target_en-de.txt')
build_alignment_input(335, 'tok.en_original_girl.en', 'hyp_original_girl.txt', 'original_girl_source-target_en-de.txt')
build_alignment_input(335, 'tok.en_original_guy.en', 'hyp_original_guy.txt', 'original_guy_source-target_en-de.txt')
build_alignment_input(335, 'tok.en_original_boy.en', 'hyp_original_boy.txt', 'original_boy_source-target_en-de.txt')


## fast_align

In [47]:
!$FAST_ALIGN -i original_man_source-target_en-de.txt -d -o -v > original_man_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_woman_source-target_en-de.txt -d -o -v > original_woman_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_girl_source-target_en-de.txt -d -o -v > original_girl_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_guy_source-target_en-de.txt -d -o -v > original_guy_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_boy_source-target_en-de.txt -d -o -v > original_boy_source-target_en-de_fast-aligned.txt

print("Finished alignment.")

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
...
expected target length = source length * 1.06441
ITERATION 1
...
  log_e likelihood: -525521
  log_2 likelihood: -758167
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.196344
       size counts: 80
ITERATION 2
...
  log_e likelihood: -82744.5
  log_2 likelihood: -119375
     cross entropy: 4.70741
        perplexity: 26.1258
      posterior p0: 0.0361041
 posterior al-feat: -0.15677
       size counts: 80
  1  model al-feat: -0.156358 (tension=4)
  2  model al-feat: -0.156561 (tension=3.99176)
  3  model al-feat: -0.156664 (tension=3.98757)
  4  model al-feat: -0.156716 (tension=3.98544)
  5  model al-feat: -0.156743 (tension=3.98436)
  6  model al-feat: -0.156756 (tension=3.98381)
  7  model al-feat: -0.156763 (tension=3.98353)
  8  model al-feat: -0.156767 (tension=3.98339)
     final tension: 3.98332
ITERATION 3
...
  log_e likelihood: -66402.1
  log_2 likelihood: -95798
     cross entro

In [48]:
import re

# Count unique translated words to the ambiguous words in translations per source sentence
def count_unique_words_alignment_translations(position, sentencesN, sourceIn, translationsIn, alignmentsIn, output):
    
    # Get positions of ambigous words
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # List with translations
    translations = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations.append(line.strip())
            
            
    
    # Extract alginments of ambiguous words
    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                #print(lineNumber)
                #print(tokens[ind])
                #print(ind)
                translated_ambiguous_words.add(tokens[ind])
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))
    
    # Add results to file
    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(translations_ambiguous_words[count]), end='\n', file=fout)
            count += 1

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        
        ############################################################
        # remove gender info; removing "in" and "e" endings in words
        set_words_new = set()
        for word in set_words:
            word_new = re.sub("in$|e$", "", word)
            #print(word_new)
            set_words_new.add(word_new)
        #print(set_words_new)
        ############################################################
        
        unique_translations += len(set_words)
        
    #print(unique_translations)
    return unique_translations/sentencesN # average

In [49]:
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_man.en', 'hyp_original_man.txt', 'original_man_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original_man.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_woman.en', 'hyp_original_woman.txt', 'original_woman_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original_woman.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_girl.en', 'hyp_original_girl.txt', 'original_girl_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original_girl.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_guy.en', 'hyp_original_guy.txt', 'original_guy_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original_guy.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_boy.en', 'hyp_original_boy.txt', 'original_boy_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original_boy.txt'))

1.6865671641791045
1.7104477611940299
1.8208955223880596
4.191044776119403
2.262686567164179


## awesome_align

In [50]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_man_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_man_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_woman_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_woman_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_girl_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_girl_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_guy_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_guy_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_boy_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_boy_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

print("Finished alignment.")

Loading the dataset...
Extracting: 3350it [00:04, 673.18it/s]
Loading the dataset...
Extracting: 3350it [00:04, 691.03it/s]
Loading the dataset...
Extracting: 3350it [00:05, 657.96it/s]
Loading the dataset...
Extracting: 3350it [00:05, 658.06it/s]
Loading the dataset...
Extracting: 3350it [00:05, 668.22it/s]
Finished alignment.


In [51]:
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_man.en', 'hyp_original_man.txt', 'original_man_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original_man.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_woman.en', 'hyp_original_woman.txt', 'original_woman_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original_woman.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_girl.en', 'hyp_original_girl.txt', 'original_girl_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original_girl.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_guy.en', 'hyp_original_guy.txt', 'original_guy_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original_guy.txt'))
print('======')
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original_boy.en', 'hyp_original_boy.txt', 'original_boy_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original_boy.txt'))

1.2328358208955223
1.2358208955223882
1.337313432835821
4.023880597014926
1.9253731343283582


# Word alignement (translation-backtranslation)

## fast_align

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [52]:
build_alignment_input(3350, 'hyp_original_man.txt', 'hyp_original_man_back.txt', 'original_man_translation-back_en-de.txt')
build_alignment_input(3350, 'hyp_original_woman.txt', 'hyp_original_woman_back.txt', 'original_woman_translation-back_en-de.txt')
build_alignment_input(3350, 'hyp_original_girl.txt', 'hyp_original_girl_back.txt', 'original_girl_translation-back_en-de.txt')
build_alignment_input(3350, 'hyp_original_guy.txt', 'hyp_original_guy_back.txt', 'original_guy_translation-back_en-de.txt')
build_alignment_input(3350, 'hyp_original_boy.txt', 'hyp_original_boy_back.txt', 'original_boy_translation-back_en-de.txt')


- Word alignement

In [53]:
!$FAST_ALIGN -i original_man_translation-back_en-de.txt -d -o -v > original_man_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_woman_translation-back_en-de.txt -d -o -v > original_woman_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_girl_translation-back_en-de.txt -d -o -v > original_girl_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_guy_translation-back_en-de.txt -d -o -v > original_guy_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i original_boy_translation-back_en-de.txt -d -o -v > original_boy_translation-back_en-de_fast-aligned.txt

print("Finished alignment.")

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................
expected target length = source length * 1.0114
ITERATION 1
.................................
  log_e likelihood: -5.23308e+06
  log_2 likelihood: -7.54974e+06
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.196092
       size counts: 178
ITERATION 2
.................................
  log_e likelihood: -795138
  log_2 likelihood: -1.14714e+06
     cross entropy: 4.54274
        perplexity: 23.3078
      posterior p0: 0.035807
 posterior al-feat: -0.159874
       size counts: 178
  1  model al-feat: -0.166873 (tension=4)
  2  model al-feat: -0.163278 (tension=4.13998)
  3  model al-feat: -0.161568 (tension=4.20806)
  4  model al-feat: -0.160727 (tension=4.24194)
  5  model al-feat: -0.160305 (tension=4.25899)
  6  model al-feat: -0.160093 (tension=4.26761)
  7  model al-feat: -0.159985 (tension=4.27199)
  8  model al-feat: -0.159931 (tension=4.27421)
     final 

- Extract target backtranslated words

In [54]:
import re

# Count unique translated words to the ambiguous words in backtranslations per source sentence
def count_unique_words_alignment_backtranslations(position_word, sentencesN, sourceIn, backtranslationsIn, alignmentsIn_translation, alignmentsIn_backtranslation, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position_word)
       
    lineNumber = 0
    counter = 0
    positions_ambiguous_words_translations = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn_translation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                positions_ambiguous_words_translations.append([int(index) for index in re.findall(regex, line)])
            else:
                positions_ambiguous_words_translations.append([999])
            lineNumber += 1
    
    # List with backtranslations
    backtranslations = []
    with open(backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip())

    lineNumber = 0
    counter = 0
    indices = []
    with open(alignmentsIn_backtranslation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            positions = positions_ambiguous_words_translations[counter] # exact positions of ambiguous words
            list_indices = []
            for position in positions:
                regex = r"" + str(position) + r"-(\d)"
                if re.findall(regex, line): 
                    list_indices.extend([int(index) for index in re.findall(regex, line)])
                else:
                    list_indices.extend([999])
            indices.append(list_indices)
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    backtranslated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        tokens = backtranslation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                #print(lineNumber)
                #print(tokens[ind])
                #print(ind)
                backtranslated_ambiguous_words.add(tokens[ind])
        lineNumber += 1
        if (lineNumber % 10 == 0):
                backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
                backtranslated_ambiguous_words = set()



    #print(backtranslations_ambiguous_words)
    print(len(backtranslations_ambiguous_words))

    # Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
    backtranslations_ambiguous_words_reduced = []
    backtranslated_ambiguous_words = set() # set forces uniqueness
    counter = 0
    for set_words in backtranslations_ambiguous_words:
        backtranslated_ambiguous_words.update(set_words)
        counter += 1
        if (counter % 10 == 0):
            backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()

    print(len(backtranslations_ambiguous_words_reduced)) 
    
    # Add results to file

    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position_word])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(backtranslations_ambiguous_words_reduced[count]), end='\n', file=fout)
            count += 1

    unique_backtranslations = 0
    for set_words in backtranslations_ambiguous_words_reduced:
        unique_backtranslations += len(set_words)
        
    return unique_backtranslations/sentencesN

In [55]:
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_man.en', 'hyp_original_man_back.txt', 'original_man_source-target_en-de_fast-aligned.txt', 'original_man_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original_man.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_woman.en', 'hyp_original_woman_back.txt', 'original_woman_source-target_en-de_fast-aligned.txt', 'original_woman_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original_woman.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_girl.en', 'hyp_original_girl_back.txt', 'original_girl_source-target_en-de_fast-aligned.txt', 'original_girl_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original_girl.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_guy.en', 'hyp_original_guy_back.txt', 'original_guy_source-target_en-de_fast-aligned.txt', 'original_guy_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original_guy.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_boy.en', 'hyp_original_boy_back.txt', 'original_boy_source-target_en-de_fast-aligned.txt', 'original_boy_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original_boy.txt'))


3350
335
8.51641791044776
3350
335
10.146268656716417
3350
335
10.053731343283582
3350
335
11.967164179104477
3350
335
11.73134328358209


## awesome_align

- Extract the position of the translated ambiguous word from each sentence

- Word alignement

In [56]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_man_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_man_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_woman_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_woman_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_girl_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_girl_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_guy_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_guy_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "original_boy_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_boy_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

print("Finished alignment.")

Loading the dataset...
Extracting: 33500it [00:37, 904.98it/s] 
Loading the dataset...
Extracting: 33500it [00:37, 894.04it/s] 
Loading the dataset...
Extracting: 33500it [00:37, 902.95it/s] 
Loading the dataset...
Extracting: 33500it [00:38, 878.11it/s] 
Loading the dataset...
Extracting: 33500it [00:38, 870.15it/s] 
Finished alignment.


- Extract target backtranslated words

In [57]:
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_man.en', 'hyp_original_man_back.txt', 'original_man_source-target_en-de_awesome-aligned.txt', 'original_man_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original_man.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_woman.en', 'hyp_original_woman_back.txt', 'original_woman_source-target_en-de_awesome-aligned.txt', 'original_woman_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original_woman.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_girl.en', 'hyp_original_girl_back.txt', 'original_girl_source-target_en-de_awesome-aligned.txt', 'original_girl_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original_girl.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_guy.en', 'hyp_original_guy_back.txt', 'original_guy_source-target_en-de_awesome-aligned.txt', 'original_guy_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original_guy.txt'))
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original_boy.en', 'hyp_original_boy_back.txt', 'original_boy_source-target_en-de_awesome-aligned.txt', 'original_boy_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original_boy.txt'))


3350
335
6.140298507462687
3350
335
8.51044776119403
3350
335
7.04179104477612
3350
335
10.82686567164179
3350
335
8.973134328358208


# Word alignement (translation-translation)

## Tercom alignement (borrowed from Tu)
- https://github.com/TuAnh23/Perturbation-basedQE/blob/master/align_and_analyse_ambiguous_trans.py#L54-L92

In [ ]:
!git clone https://github.com/TuAnh23/Perturbation-basedQE.git

In [58]:
%cd $TERCOM

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE


In [59]:
import align_and_analyse_ambiguous_trans as tercom
import pandas as pd

def count_unique_words_tercom_alignment(position, sentencesN, sourceIn, backtranslationsIn):
    # List with source sentences; output 100 times to match backtranslation size
    source = []
    with open(PATH + "/" + sourceIn, 'r') as fin:
        for line in fin:
            for i in range(100): # append the source sentence 100 times to match backtranslations later
                source.append(line.strip().split()) # split() tokenizes the sentence, because tercom expects tokens     

    print(len(source))

    # List with original backtranslations
    backtranslations = []
    with open(PATH + "/" + backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip().split())

    print(len(backtranslations))
    
    # Generate alignments
    alignments = tercom.tercom_alignment(source, backtranslations)
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # Extract target translated words to source words
    lineNumber = 0
    counter = 0
    indices = []
    for align in alignments:
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        indices.append([item[1] for item in (item for item in align if not(pd.isna(item[0]))) if item[0] == position][0])
        lineNumber += 1
        if (lineNumber % 100 == 0):
            counter += 1

    print(len(indices))

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        backtranslation_index = backtranslations.index(backtranslation)
        if not(pd.isna(indices[backtranslation_index])):
            translated_ambiguous_words.add(backtranslation[indices[backtranslation_index]])
        lineNumber += 1
        if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        unique_translations += len(set_words)
        
    return unique_translations/sentencesN

In [60]:
print(count_unique_words_tercom_alignment(1, 335, 'tok.en_original_man.en', 'hyp_original_man_back.txt'))
print(count_unique_words_tercom_alignment(1, 335, 'tok.en_original_woman.en', 'hyp_original_woman_back.txt'))
print(count_unique_words_tercom_alignment(1, 335, 'tok.en_original_girl.en', 'hyp_original_girl_back.txt'))
print(count_unique_words_tercom_alignment(1, 335, 'tok.en_original_guy.en', 'hyp_original_guy_back.txt'))
print(count_unique_words_tercom_alignment(1, 335, 'tok.en_original_boy.en', 'hyp_original_boy_back.txt'))


33500
33500
33500
10.501492537313434
33500
33500
33500
12.385074626865672
33500
33500
33500
9.764179104477613
33500
33500
33500
17.883582089552238
33500
33500
33500
13.071641791044776


In [61]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity/Sampling/Masking


# Word occurrence

## Translation

In [62]:
def extract_word_translations(filename_tokenized, filename_translations, filename_out, filename_alignments):
    """
    Match alignment indices from translation to backtranslation
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_alignments, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 335:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [63]:
def count_word_translations(filename_tokenized, filename_translations, filename_out, filename_alignments):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_alignments, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0
    occurrences = []
    with open(filename_out, 'w') as fout:
        while count < 335:
            occurrences.append([len(target_set) for target_set in target_words[count]])
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1
    
    return occurrences

- Investigate the variability of the remaining sentence without the ambiguos word

In [64]:
def uniqueness_rest_of_sentence(position, sentencesN, occurrences):
    
    # Get positions of ambigous words
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # Sum over all sentences and build average     
    counter = 0
    sum_sent = 0
    for occur in occurrences:
        position = positions_ambiguous_words[counter]
        sum_sent += (sum(occur) - occur[position])/(len(occur) - 1) # sum for every sentence
        counter += 1
        
    return sum_sent/sentencesN

In [65]:
extract_word_translations('tok.en_original_man.en', 'hyp_original_man.txt', 'translations_words_original_man.txt', 'original_man_source-target_en-de_awesome-aligned.txt')
occurrence_original_man = count_word_translations('tok.en_original_man.en', 'hyp_original_man.txt', 'translations_words_original_man_occurrence.txt', 'original_man_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(1, 335, occurrence_original_man))

extract_word_translations('tok.en_original_woman.en', 'hyp_original_woman.txt', 'translations_words_original_woman.txt', 'original_woman_source-target_en-de_awesome-aligned.txt')
occurrence_original_woman = count_word_translations('tok.en_original_woman.en', 'hyp_original_woman.txt', 'translations_words_original_woman_occurrence.txt', 'original_woman_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(1, 335, occurrence_original_woman))

extract_word_translations('tok.en_original_girl.en', 'hyp_original_girl.txt', 'translations_words_original_girl.txt', 'original_girl_source-target_en-de_awesome-aligned.txt')
occurrence_original_girl = count_word_translations('tok.en_original_girl.en', 'hyp_original_girl.txt', 'translations_words_original_girl_occurrence.txt', 'original_girl_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(1, 335, occurrence_original_girl))

extract_word_translations('tok.en_original_guy.en', 'hyp_original_guy.txt', 'translations_words_original_guy.txt', 'original_guy_source-target_en-de_awesome-aligned.txt')
occurrence_original_guy = count_word_translations('tok.en_original_guy.en', 'hyp_original_guy.txt', 'translations_words_original_guy_occurrence.txt', 'original_guy_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(1, 335, occurrence_original_guy))

extract_word_translations('tok.en_original_boy.en', 'hyp_original_boy.txt', 'translations_words_original_boy.txt', 'original_boy_source-target_en-de_awesome-aligned.txt')
occurrence_original_boy = count_word_translations('tok.en_original_boy.en', 'hyp_original_boy.txt', 'translations_words_original_boy_occurrence.txt', 'original_boy_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(1, 335, occurrence_original_boy))


2.4544563849963326
2.436792330681706
2.4161079691940084
2.2069089996921445
2.3505158065035157


## Backtranslation

In [66]:
def extract_alignment_indices_backtranslation(filename_translations, filename_backtranslations):
    """
    Extract alignment indices
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_translations, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
       
    # Match alignement indices from translation to backtranslation
    lineNumber = 0
    counter = 0
    indices_backtranslation = []
    with open(filename_backtranslations, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            alignement_tokens = line.split()
            indices_line = []
            for index_list in indices_translation[counter]:
                index_matches = []
                for index in index_list:
                    regex = r"" + str(index) + r"-(\d)"
                    if re.findall(regex, line): 
                        index_matches.extend([int(i) for i in re.findall(regex, line)])
                    else:
                        index_matches.extend([999])
                indices_line.append(index_matches)
            indices_backtranslation.append(indices_line)
            lineNumber += 1 
    return indices_backtranslation

In [67]:
def extract_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 335:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [68]:
def count_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())
       
    count = 0
    occurrences = []
    with open(filename_out, 'w') as fout:
        while count < 335:
            occurrences.append([len(target_set) for target_set in target_words[count]])
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1
    
    return occurrences

In [69]:
indices_original_man = extract_alignment_indices_backtranslation('original_man_source-target_en-de_awesome-aligned.txt', 'original_man_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original_man.en', 'hyp_original_man_back.txt', 'backtranslations_words_original_man.txt', indices_original_man)
occurrences_original_man = count_word_backtranslations('tok.en_original_man.en', 'hyp_original_man_back.txt', 'backtranslations_words_original_man_occurrence.txt', indices_original_man)
print(uniqueness_rest_of_sentence(1, 335, occurrences_original_man))

indices_original_woman = extract_alignment_indices_backtranslation('original_woman_source-target_en-de_awesome-aligned.txt', 'original_woman_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original_woman.en', 'hyp_original_woman_back.txt', 'backtranslations_words_original_woman.txt', indices_original_woman)
occurrences_original_woman = count_word_backtranslations('tok.en_original_woman.en', 'hyp_original_woman_back.txt', 'backtranslations_words_original_woman_occurrence.txt', indices_original_woman)
print(uniqueness_rest_of_sentence(1, 335, occurrences_original_woman))

indices_original_girl = extract_alignment_indices_backtranslation('original_girl_source-target_en-de_awesome-aligned.txt', 'original_girl_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original_girl.en', 'hyp_original_girl_back.txt', 'backtranslations_words_original_girl.txt', indices_original_girl)
occurrences_original_girl = count_word_backtranslations('tok.en_original_girl.en', 'hyp_original_girl_back.txt', 'backtranslations_words_original_girl_occurrence.txt', indices_original_girl)
print(uniqueness_rest_of_sentence(1, 335, occurrences_original_girl))

indices_original_guy = extract_alignment_indices_backtranslation('original_guy_source-target_en-de_awesome-aligned.txt', 'original_guy_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original_guy.en', 'hyp_original_guy_back.txt', 'backtranslations_words_original_guy.txt', indices_original_guy)
occurrences_original_guy = count_word_backtranslations('tok.en_original_guy.en', 'hyp_original_guy_back.txt', 'backtranslations_words_original_guy_occurrence.txt', indices_original_guy)
print(uniqueness_rest_of_sentence(1, 335, occurrences_original_guy))

indices_original_boy = extract_alignment_indices_backtranslation('original_boy_source-target_en-de_awesome-aligned.txt', 'original_boy_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original_boy.en', 'hyp_original_boy_back.txt', 'backtranslations_words_original_boy.txt', indices_original_boy)
occurrences_original_boy = count_word_backtranslations('tok.en_original_boy.en', 'hyp_original_boy_back.txt', 'backtranslations_words_original_boy_occurrence.txt', indices_original_boy)
print(uniqueness_rest_of_sentence(1, 335, occurrences_original_boy))

8.840463287078734
8.58198432335395
8.698465572570047
7.862175232824928
8.053851990366475


# Gender statistics

In [70]:
from enum import Enum

class GENDER(Enum):
    """
    Enumerate possible genders.
    Ignore option resolves to words that should be ignored in particular language
    """
    male = 0
    female = 1
    neutral = 2
    unknown = 3
    ignore = 4
    
# ??? These are not always correct; 'der' could be Dativ or Genitiv for female, 'die' could be plural
# !!! There isn't always an article
DE_DETERMINERS = {"der": GENDER.male, "ein": GENDER.male, "dem": GENDER.male, "den": GENDER.male, 
                  "einen": GENDER.male, "des": GENDER.male, "er": GENDER.male, "seiner": GENDER.male,
                  "ihn": GENDER.male, "seinen": GENDER.male, "ihm": GENDER.male, "ihren": GENDER.male,
                  "die": GENDER.female, "eine": GENDER.female, "einer": GENDER.female, "seinem": GENDER.male,
                  "ihrem": GENDER.male, "sein": GENDER.male,
                  "sie": GENDER.female, "seine": GENDER.female, "ihrer": GENDER.female, 
                  "ihr": GENDER.neutral, "ihre": GENDER.neutral, "das": GENDER.neutral,
                  "jemanden": GENDER.neutral}

def get_german_determiners(words):
    """
    Get a list of (gender)
    given a list of words.
    """
    determiners = []
    for (word_ind, word) in enumerate(words):
        word = word.lower()
        if word in DE_DETERMINERS:
            determiners.append((DE_DETERMINERS[word].name))
    return determiners

In [71]:
dets = get_german_determiners(["dem"])
print(dets)

['male']


- Calculate gender based on the articles of unique words: how many of the sentences produce both genders, female and male

In [72]:
import re

# Extract articles of target tranlsated words
def extract_articles(position, sentencesN, translationsIn, alignmentsIn, sourceIn, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
    
    # List with original translations
    translations_original = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations_original.append(line.strip())


    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations_original:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                translated_ambiguous_words.add(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()


    #print(len(translations_ambiguous_words))
    
    # Add results to file

    # List with original source sentences
    source = []
    ambiguous_words = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0  
    genders = []
    male = []
    female = []
    with open(output, 'w') as fout:
        while count < sentencesN:
            #print(translations_ambiguous_words[count])
            genders.append(set(get_german_determiners(translations_ambiguous_words[count])))
            male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
            female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
            count += 1
            
    return (sum(1 for i in genders if ('male' in i and 'female' in i)), 
            male.count(True), female.count(True))

In [73]:
print(extract_articles(1, 335, 'hyp_original_man.txt', 'original_man_source-target_en-de_awesome-aligned.txt', 'tok.en_original_man.en', 'unique-words_translations_original_man_articles.txt'))
print(extract_articles(1, 335, 'hyp_original_woman.txt', 'original_woman_source-target_en-de_awesome-aligned.txt', 'tok.en_original_woman.en', 'unique-words_translations_original_woman_articles.txt'))
print(extract_articles(1, 335, 'hyp_original_girl.txt', 'original_girl_source-target_en-de_awesome-aligned.txt', 'tok.en_original_girl.en', 'unique-words_translations_original_girl_articles.txt'))
print(extract_articles(1, 335, 'hyp_original_guy.txt', 'original_guy_source-target_en-de_awesome-aligned.txt', 'tok.en_original_guy.en', 'unique-words_translations_original_guy_articles.txt'))
print(extract_articles(1, 335, 'hyp_original_boy.txt', 'original_boy_source-target_en-de_awesome-aligned.txt', 'tok.en_original_boy.en', 'unique-words_translations_original_boy_articles.txt'))


(12, 335, 12)
(9, 9, 335)
(4, 15, 73)
(14, 335, 14)
(11, 335, 11)


- Calculate gender in percentage for each sentence: percent of "male" vs. female in translations for each sentence

In [74]:
import re

# Extract articles of target tranlsated words
def extract_articles_percent(position, sentencesN, translationsIn, alignmentsIn, sourceIn, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
    
    # List with original translations
    translations_original = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations_original.append(line.strip())


    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = [] 
    for translation in translations_original:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                translated_ambiguous_words.append(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = []


    #print(len(translations_ambiguous_words))
    
    # Add results to file

    # List with original source sentences
    source = []
    ambiguous_words = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0  
    genders = []
    male = []
    female = []
    with open(output, 'w') as fout:
        while count < sentencesN:
            #print(translations_ambiguous_words[count])
            genders.append(get_german_determiners(translations_ambiguous_words[count]))
            male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
            female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
            count += 1
     
    #print(genders)
    return (sum([i.count('male')/10 for i in genders])/sentencesN*100, 
            sum([i.count('female')/10 for i in genders])/sentencesN*100)

In [75]:
print(extract_articles_percent(1, 335, 'hyp_original_man.txt', 'original_man_source-target_en-de_awesome-aligned.txt', 'tok.en_original_man.en', 'unique-words_translations_original_man_articles.txt'))
print(extract_articles_percent(1, 335, 'hyp_original_woman.txt', 'original_woman_source-target_en-de_awesome-aligned.txt', 'tok.en_original_woman.en', 'unique-words_translations_original_woman_articles.txt'))
print(extract_articles_percent(1, 335, 'hyp_original_girl.txt', 'original_girl_source-target_en-de_awesome-aligned.txt', 'tok.en_original_girl.en', 'unique-words_translations_original_girl_articles.txt'))
print(extract_articles_percent(1, 335, 'hyp_original_guy.txt', 'original_guy_source-target_en-de_awesome-aligned.txt', 'tok.en_original_guy.en', 'unique-words_translations_original_guy_articles.txt'))
print(extract_articles_percent(1, 335, 'hyp_original_boy.txt', 'original_boy_source-target_en-de_awesome-aligned.txt', 'tok.en_original_boy.en', 'unique-words_translations_original_boy_articles.txt'))


(89.9104477611941, 0.417910447761194)
(0.44776119402985076, 92.95522388059703)
(0.6268656716417912, 2.7164179104477584)
(85.0447761194031, 0.5970149253731344)
(91.31343283582099, 0.5373134328358209)
